In [1]:
import tensorflow as tf
import numpy as np


In [2]:
def pretty_print_guess_tensor(const_guess, operand_guess, operator_guess):
    # TODO: const_guess

    s = []

    for t in operand_guess:
        s += [f'x_{tf.argmax(t)}']

    operator_lookup = ['+','-', '*','/']
    result = s[::]
    for i, op_one_hot in enumerate(operator_guess):
        operators = tf.argmax(op_one_hot,axis=-1)
        left = result[::2]
        right = (result[1:] + result[:1])[::2]
        ops = operators[:len(left)]
        result = []
        for l, op, r in zip(left, ops, right):
            result += [f'({l} {operator_lookup[op]} {r})']


    return ' '.join(result)

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)

pretty_print_guess_tensor(const_guess, operand_guess, operator_guess)

'(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))'

In [3]:
@tf.function
def to_prob_dist_all(v):
    v2 = tf.sqrt(tf.square(v)+1e-9)
    # v2 = tf.sqrt(tf.square(v))
    m = tf.expand_dims(tf.reduce_sum(v2, axis=-1),-1)
    n = tf.math.divide_no_nan(v2, m)
    return n

tf.print(tf.argmax(operator_guess))
tf.print(tf.argmax(to_prob_dist_all(operator_guess)))

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [4]:
@tf.function
def cross_entropy(x, y, epsilon = 1e-9):
    return -2 * tf.reduce_mean(y * tf.math.log(x + epsilon), -1) / tf.math.log(2.)

x = tf.constant([
    [1.0,0],
    [0.5,0.5],
    [.75,.25]
    ]
,dtype=tf.float32)

with tf.GradientTape() as tape:
    tape.watch(x)
    y = cross_entropy(x, x)

tf.print(y)
tf.print(tape.gradient(y, x))

[-0 1 0.811278105]
[[-1.44269502 29.8973541]
 [-0.442695022 -0.442695022]
 [-1.02765751 0.557305]]


In [5]:
@tf.function
def entropy(x):
    return cross_entropy(x, x)

In [6]:
@tf.function
def dot(x, y):
    r = tf.multiply(x, y)
    return tf.reduce_sum(r, -1)

x = tf.constant([
    [2,2,2],
    [3,3,3]
])

dot(x, x)

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([12, 27])>

In [7]:
@tf.function
def resolve_values(const_guess, values, operand_guess):
    # TODO: const_guess

    operand_guess = to_prob_dist_all(operand_guess)

    operand_count = tf.shape(operand_guess)[0]
    values = tf.expand_dims(values, axis=0)
    values = tf.tile(values, [operand_count,1])
    
    result = dot(values, operand_guess)

    return result

v1 = tf.range(NUM_LEAVES)
cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
# operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
operand_guess = tf.constant([
       [ 0,  0,  0,  0,  0, 1.0, 0, 0],
       [ 0,  0,  0,  0,  0, 1.0, 0, 0],
       [ 0,  0,  0,  0,  0, 1.0, 0, 0],
       [ 0,  0,  0,  0,  0, 1.0, 0, 0],
       [ 0,  0,  0,  0,  0, 1.0, 0, 0],
       [ 0,  0,  0,  0,  0, 1.0, 0, 0],
       [ 0,  0,  0,  0,  0, 1.0, 0, 0],
       [ 0,  0,  0,  0,  0, 1.0, 0, 0],
])
operand_guess = tf.Variable(operand_guess)
# operand_guess = tf.Variable([
#     0,0.5,0.5,0, 0,0,0,0,
# ],dtype=tf.float32)
values = tf.cast(v1,dtype=tf.float32) + 1.0

target = tf.ones((NUM_LEAVES,), dtype=tf.float32) * 2.0

# opt = tf.keras.optimizers.Adam(3e-4)
opt = tf.keras.optimizers.Adam(1e-2)
resolved = resolve_values(const_guess, values, operand_guess)
for i in range(1000):
    with tf.GradientTape() as tape:
        resolved = resolve_values(const_guess, values, operand_guess)

        target_loss = tf.nn.l2_loss(resolved - target)
        entropy_loss = entropy(operand_guess)

        loss = target_loss + entropy_loss * 1e+1
    
    variables = [operand_guess]
    grads = tape.gradient(loss, variables)
    opt.apply_gradients(zip(grads, variables))

    operand_guess.assign(to_prob_dist_all(operand_guess))

    if i % 100 == 0:
        print_idx = 2
        dist = list(tf.round(operand_guess * 100).numpy().astype(np.int32))
        tf.print(resolved[print_idx], loss[print_idx], target_loss, entropy_loss[print_idx], dist[print_idx])

np.vstack(tf.round(operand_guess * 100).numpy().astype(np.int32))

5.99962091 63.9878693 63.9878693 -0 array([ 1,  1,  1,  1,  1, 94,  1,  1])
1.8825922 4.25354767 0.0551383644 0.419840902 array([40, 35, 23,  0,  0,  1,  0,  0])
1.84156811 4.22222662 0.100402653 0.412182391 array([41, 37, 20,  0,  0,  1,  1,  0])
1.77605987 4.15201044 0.200596735 0.395141363 array([42, 42, 15,  0,  0,  1,  1,  0])
1.61463773 3.40017581 0.594016314 0.280615956 array([42, 56,  1,  0,  0,  1,  0,  0])
1.73385024 2.85548139 0.283342779 0.257213861 array([30, 66,  1,  0,  0,  1,  1,  0])
1.90067697 1.96043122 0.0394602604 0.192097098 array([14, 81,  1,  0,  0,  1,  1,  1])
2.07097793 0.782594442 0.0201514643 0.0762443 array([ 0, 96,  1,  0,  0,  1,  1,  0])
2.10031509 0.911767364 0.0402524732 0.0871514902 array([ 0, 97,  1,  0,  0,  1,  1,  0])
2.10497761 0.913960576 0.0440811925 0.086987935 array([ 0, 97,  1,  0,  0,  1,  1,  0])


array([[ 1, 96,  0,  1,  1,  0,  0,  1],
       [ 1, 96,  0,  1,  1,  0,  0,  1],
       [ 1, 96,  0,  1,  1,  0,  0,  1],
       [ 1, 96,  0,  1,  1,  0,  0,  1],
       [ 1, 96,  0,  1,  1,  0,  0,  1],
       [ 1, 96,  0,  1,  1,  0,  0,  1],
       [ 1, 96,  0,  1,  1,  0,  0,  1],
       [ 1, 96,  0,  1,  1,  0,  0,  1]])

In [8]:
@tf.function
def operate(operands, operators):
    left = operands[::2]
    right = tf.roll(operands, shift=-1, axis=0)[::2]

    r_add = left + right
    r_sub = left - right
    r_mul = left * right
    r_div = tf.math.divide_no_nan(left, right)

    r = tf.stack([r_add, r_sub, r_mul, r_div], axis=1)

    operators = to_prob_dist_all(operators)

    return dot(r, operators)

operands = tf.range(NUM_LEAVES, dtype=tf.float32)
v2 = tf.range(NUM_OPERATORS)
operators = tf.constant([
    [1,0,0,0],
    [0,1,0,0],
    [0,0,1,0],
    [0,0,0,1],
],dtype=tf.float32)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(operands)
    tape.watch(operators)
    result = operate(operands, operators)

tf.print(result)
tf.print(tf.reshape(tape.gradient(result, operands),(2,4)))
tf.print(tape.gradient(result, operators))

[0.999873519 -0.999536216 19.9983807 0.858769]
[[0.99999994 0.999873519 1.00004208 -0.999817252]
 [4.99959469 3.99961543 0.143128172 -0.122247636]]
[[0.000126481056 0 0 0]
 [0 -0.000463724136 0 0]
 [0 0 0.00161933899 0]
 [0 0 0 -0.0016259551]]


In [9]:
def eager_process_block(operands, operators_arr):
    acc = operands

    for operators in operators_arr:
        num_operands = tf.shape(acc)[0]
        operators = operators[:num_operands // 2]
        acc = operate(acc, operators)

    return acc

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)
operands = resolve_values(const_guess, values, operand_guess)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(operands)
    tape.watch(operator_guess)
    result = eager_process_block(operands, operator_guess)

tf.print(pretty_print_guess_tensor(const_guess, operand_guess, operator_guess))
x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7 = list(range(8))
tf.print((((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7))))
tf.print(result)
tf.print(tf.reshape(tape.gradient(result, operands),(2,4)))
tf.print(tape.gradient(result, operator_guess))

(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
19.142857142857142
[19.1372566]
[[1.00041687 1.00029039 1.00045919 -1.00023425]
 [4.99858665 3.99898624 -0.143142194 0.122262307]]
[[[0.000126719475 0 0 0]
  [0 -0.000464081764 0 0]
  [0 0 0.00161933899 0]
  [0 0 0 0.00162553787]]

 [[2.67056748e-07 0 0 0]
  [0 -0.000123977661 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0.00241851807 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]


In [10]:
@tf.function
def unrolled_process_block_3(operands, operators_arr):
    acc = operands

    # Level 1
    operators = operators_arr[0]
    operators = operators[:4]
    acc = operate(acc, operators)

    # Level 2
    operators = operators_arr[1]
    operators = operators[:2]
    acc = operate(acc, operators)

    # Level 3
    operators = operators_arr[2]
    operators = operators[:1]
    acc = operate(acc, operators)

    return acc

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)
operands = resolve_values(const_guess, values, operand_guess)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(operands)
    tape.watch(operator_guess)
    result = unrolled_process_block_3(operands, operator_guess)

tf.print(pretty_print_guess_tensor(const_guess, operand_guess, operator_guess))
x_0, x_1, x_2, x_3, x_4, x_5, x_6, x_7 = list(range(8))
tf.print((((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7))))
tf.print(result)
tf.print(tf.reshape(tape.gradient(result, operands),(2,4)))
tf.print(tape.gradient(result, operator_guess))

(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))
19.142857142857142
[19.1372566]
[[1.00041687 1.00029039 1.00045919 -1.00023425]
 [4.99858665 3.99898624 -0.143142194 0.122262307]]
[[[0.000126719475 0 0 0]
  [0 -0.000464081764 0 0]
  [0 0 0.00161933899 0]
  [0 0 0 0.00162553787]]

 [[2.67056748e-07 0 0 0]
  [0 -0.000123977661 0 0]
  [0 0 0 0]
  [0 0 0 0]]

 [[0.00241851807 0 0 0]
  [0 0 0 0]
  [0 0 0 0]
  [0 0 0 0]]]


In [11]:
def print_collapsed_result(values, const_guess, operand_guess, operator_guess):
    # TODO: const_guess

    operands = tf.round(operand_guess)
    acc = resolve_values(const_guess, values, operands)
    operators = tf.round(operator_guess)

    result = eager_process_block(acc, operators)

    return result

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)

print_collapsed_result(values, const_guess, operand_guess, operator_guess)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([19.137257], dtype=float32)>

In [12]:
def bind_opt_train_step(opt, entropy_weight=1e+2):
    @tf.function
    def train_step(const_guess, operand_guess, operator_guess, values, target):
        with tf.GradientTape() as tape:
            cg, opg, otg = const_guess, operand_guess, operator_guess
            # cg = tf.nn.softmax(cg)
            # opg = tf.nn.softmax(opg)
            # otg = tf.nn.softmax(otg)

            cg = to_prob_dist_all(cg)
            opg = to_prob_dist_all(opg)
            otg = to_prob_dist_all(otg)

            cg_entropy = 0.0 # TODO
            opg_entropy = tf.reduce_sum(entropy(opg))
            otg_entropy = tf.reduce_sum(entropy(otg))

            operands = resolve_values(cg, values, opg)
            result = unrolled_process_block_3(operands, otg)

            target_loss = tf.nn.l2_loss(result[0] - target)

            loss = target_loss

            if target_loss < 1:
                loss += entropy_weight * (opg_entropy + otg_entropy)

        variables = [operand_guess, operator_guess]
        grads = tape.gradient(loss, variables)
        # grads = [tf.clip_by_norm(g, 100.0) for g in grads]
        opt.apply_gradients(zip(grads, variables))

        const_guess.assign(to_prob_dist_all(const_guess))
        operand_guess.assign(to_prob_dist_all(operand_guess))
        operator_guess.assign(to_prob_dist_all(operator_guess))

        return loss, target_loss, cg_entropy, opg_entropy, otg_entropy

    return train_step

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)

const_guess = tf.Variable(const_guess)
operand_guess = tf.Variable(operand_guess)
operator_guess = tf.Variable(operator_guess)

target = 19.0

opt = tf.keras.optimizers.Adam(3e-4)
# opt = tf.keras.optimizers.SGD(1e-1)
train_step = bind_opt_train_step(opt)

with tf.GradientTape(persistent=True) as tape:
    tape.watch(const_guess)
    tape.watch(operand_guess)
    tape.watch(operator_guess)

    result = train_step(const_guess, operand_guess, operator_guess, values, target)

tf.print(result)
tf.print(tape.gradient(result, operand_guess))
tf.print(tape.gradient(result, operator_guess))

(1.66757882, 0.00910403579, 0, 0.00725521892, 0.00932952948)
[[-0.0836830139 0 0 ... 0 0 0]
 [0 -0.0836334229 0 ... 0 0 0]
 [0 0 -0.0835838318 ... 0 0 0]
 ...
 [0 0 0 ... -0.0832290649 0 0]
 [0 0 0 ... 0 -0.083530426 0]
 [0 0 0 ... 0 0 -0.0834960938]]
[[[-0.0715789795 0 0 0]
  [0 -0.0716934204 0 0]
  [0 0 -0.0712966919 0]
  [0 0 0 -0.0712966919]]

 [[-0.0716018677 0 0 0]
  [0 -0.0716323853 0 0]
  [0 0 -0.0716095 0]
  [0 0 0 -0.0716095]]

 [[-0.071144104 0 0 0]
  [0 -0.0716095 0 0]
  [0 0 -0.0716095 0]
  [0 0 0 -0.0716095]]]


In [13]:
# opt = tf.keras.optimizers.Adam(3e-4)
opt = tf.keras.optimizers.Adam(1e-2)
# opt = tf.keras.optimizers.SGD(1e-1)
train_step = bind_opt_train_step(opt, 1)

NUM_LEAVES = 8
NUM_OPERATORS = 4
v1 = tf.range(NUM_LEAVES)
v2 = tf.range(NUM_OPERATORS)

cgv = tf.one_hot(v1 // 2, NUM_LEAVES//2, dtype=tf.float32)
const_guess = tf.concat([cgv, cgv],axis=1)
operand_guess = tf.one_hot(v1, NUM_LEAVES, dtype=tf.float32)
ogv = tf.expand_dims(tf.one_hot(v2, NUM_OPERATORS, dtype=tf.float32), axis=0)
operator_guess = tf.concat([ogv,ogv,ogv], axis=0)
values = tf.cast(v1,dtype=tf.float32)

const_guess = tf.Variable(const_guess)
operand_guess = tf.Variable(operand_guess)
operator_guess = tf.Variable(operator_guess)

target = 7.0
steps = 1000
for i in range(steps):
    loss, target_loss, cg_entropy, opg_entropy, otg_entropy = train_step(const_guess, operand_guess, operator_guess, values, target)

    if i % (steps // 10) == 0:
        cg = const_guess.numpy()
        opg = operand_guess.numpy()
        otg = operator_guess.numpy()
        collapsed_result = print_collapsed_result(values, cg, opg, otg)

        tf.print(i, collapsed_result[0], loss, target_loss, cg_entropy, opg_entropy, otg_entropy)
        # tf.print(pretty_print_guess_tensor(cg, opg, otg))
tf.print(pretty_print_guess_tensor(cg, opg, otg))

0 19.1372566 73.6283493 73.6283493 0 0.00725521892 0.00932952948
100 19.1372566 2.81111312 0.00345588336 0 1.18885708 1.61880016
200 19.1372566 2.19650316 0.00878225453 0 0.898448706 1.28927231
300 19.1372566 1.9347347 7.87916069e-05 0 0.862380922 1.07227492
400 19.1372566 1.96360183 0.104608402 0 0.793349147 1.06564426
500 19.1372566 1.87586272 0.00224292884 0 0.894468546 0.979151189
600 19.1372566 1.78837192 0.043634709 0 0.872879863 0.871857405
700 19.1372566 1.82108438 0.0674607903 0 0.917027 0.836596608
800 19.1372566 2.20998025 0.530530274 0 0.905871391 0.773578525
900 19.1372566 2.08362675 0.372966111 0 0.917841613 0.792819
(((x_0 + x_1) + (x_2 - x_3)) + ((x_4 * x_5) - (x_6 / x_7)))


In [14]:
tf.round(opg * 100)


<tf.Tensor: shape=(8, 8), dtype=float32, numpy=
array([[98.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
       [ 0., 98.,  1.,  0.,  1.,  0.,  0.,  0.],
       [ 1.,  0., 98.,  1.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0., 98.,  0.,  1.,  0.,  0.],
       [23.,  0.,  0.,  0., 76.,  0.,  0.,  0.],
       [33., 10.,  0.,  0.,  1., 56.,  0.,  0.],
       [ 1.,  1.,  0.,  1.,  0.,  0., 97.,  1.],
       [ 1.,  1.,  1.,  0.,  0.,  0.,  1., 96.]], dtype=float32)>

In [15]:
tf.round(entropy(opg)*100)

<tf.Tensor: shape=(8,), dtype=float32, numpy=array([ 5.,  5.,  4.,  6., 22., 35.,  7.,  7.], dtype=float32)>